## SRJ

In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []
grad_old=[]

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 3.0
symbols = ["H","H","H","H"]
electrons = 4
orbitals = 8
r_bohr = bond_length *1.8897259886  
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")
hf_state = qchem.hf_state(electrons, qubits)




def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        print('Updated hf_state is', hf_state)  
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    def callback(params):
        print(f"Current parameters: {params}")
        print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    states = [hf_state]
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
            #print(f'The expectation value of {i} is', current_value)

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        #print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value
        grad_old.append(max_value)
        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
        if j == 0:
            params = qml.numpy.zeros(len(ash_excitation), requires_grad=True)  #Parameters initialization
            #print('Parameters are', params)
        else:
            params = np.append(params, 0.0) 
            #print('Parameters before optimization are', params)


        #Energy calculation
        result = minimize(cost, params, method='powell', callback=callback, tol = e_th, options = {'disp': True, 'maxiter': 1e8, 'xtol':1e-8, 'ftol':1e-8})

        #print("Final updated parameters:", result.x)
        #print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)
        print('Updated params are', params)
        print('Updated excitation are', ash_excitation)

        # New state generation - With the updated parameters
        ostate = new_state(hf_state, ash_excitation, params)
        #print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        # Append the new state to the states list
        states.append(ostate)

    return energies, params, ash_excitation, gs_state, H, qubits, grad_old

In [2]:
energies, params, ash_excitation, gs_state,H, qubits, grad_old = adaptvqe(adapt_it = 5, e_th=1e-12)
print('Energies:', energies)
print('Gradients:', grad_old)

Updated hf_state is [1 1 1 1 0 0 0 0]
HF state is -1.332568888066893
The adapt iteration now is 0


/Users/s.poyyapakkam/miniconda3/envs/ashsri/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


Highest gradient excitation is [2, 3, 6, 7]
Current parameters: [1.12299215]
Current cost: -1.4300836708403135

Current parameters: [1.12299279]
Current cost: -1.4300836708403395

Optimization terminated successfully.
         Current function value: -1.430084
         Iterations: 2
         Function evaluations: 79
Updated params are [1.12299279]
Updated excitation are [[2, 3, 6, 7]]
The adapt iteration now is 1
Highest gradient excitation is [0, 3, 5, 6]
Current parameters: [1.12299268 1.38120324]
Current cost: -1.5693729295710581

Current parameters: [1.09743885 1.37779486]
Current cost: -1.5694603734962098

Current parameters: [1.09716182 1.37775789]
Current cost: -1.5694603835520702

Optimization terminated successfully.
         Current function value: -1.569460
         Iterations: 3
         Function evaluations: 203
Updated params are [1.09716182 1.37775789]
Updated excitation are [[2, 3, 6, 7], [0, 3, 5, 6]]
The adapt iteration now is 2
Highest gradient excitation is [0, 1, 4

## Measurement of states

In [ ]:
import pennylane as qml

# Initialize device with 1000 shots
dev4 = qml.device("lightning.qubit", wires=qubits, shots=1000)

@qml.qnode(dev4)
def measurement(gs_state):
    qml.StatePrep(gs_state, wires=range(qubits))
    return qml.counts(all_outcomes = False)  # Samples all qubits

counts = measurement(gs_state)
print(counts)
#Next highest state : |00001111> - 213

{'00001111': 213, '00111100': 116, '01001011': 7, '01100110': 139, '01111000': 5, '10000111': 1, '10011001': 120, '10110100': 9, '11000011': 119, '11110000': 271}


## Taking second highest state - |00001111>

In [29]:
import pennylane as qml
import numpy as np

dev = qml.device("lightning.qubit", wires=8)

@qml.qnode(dev)
def givens_circuit(theta1, theta2):
    qml.BasisEmbedding(hf_state, wires=range(qubits))

    qml.DoubleExcitation(theta1, wires=[2,3,4,5])  # Generalized Givens rotation
    qml.DoubleExcitation(theta2, wires=[0,1,6,7])
    #qml.ctrl(qml.DoubleExcitation, control=2)(theta1, wires=[0,1,4,5])
    #qml.ctrl(qml.DoubleExcitation, control=1)(theta2, wires=[2,3,6,7])
    return qml.state()

#qml.ctrl(qml.SingleExcitation, control=1)(params[2], wires=[0, 2])
theta1= np.pi/2
theta2 = np.pi/2
comb_state = givens_circuit(theta1, theta2)
#print(f"State: {comb_state}")
qubits=8

dev4 = qml.device("lightning.qubit", wires=qubits, shots=1000)
@qml.qnode(dev4)
def measurement(comb_state):
    qml.StatePrep(comb_state, wires=range(qubits))
    return qml.counts(all_outcomes = False)  # Samples all qubits

counts = measurement(comb_state)
print(counts)

{'00001111': 252, '00110011': 244, '11001100': 242, '11110000': 262}


In [33]:
import pennylane as qml
import numpy as np

qubits = 8
hf_state = np.array([1, 1, 1, 1, 0, 0, 0, 0])  # |11110000⟩

dev = qml.device("lightning.qubit", wires=qubits)

@qml.qnode(dev)
def givens_circuit(theta1, theta2):
    qml.BasisEmbedding(hf_state, wires=range(qubits))
    
    # Apply first DoubleExcitation (unconditional)
    qml.DoubleExcitation(theta1, wires=[0, 1, 4, 5])
    
    # Apply second DoubleExcitation CONDITIONALLY
    # Control on qubits 0 and 1 being |00⟩ (after first swap)
    qml.ctrl(
        qml.DoubleExcitation,
        control=[0, 1],
        control_values="00"
    )(theta2, wires=[2, 3, 6, 7])
    
    return qml.state()

theta1 = np.pi  # Full swap for deterministic behavior
theta2 = np.pi
comb_state = givens_circuit(theta1, theta2)

# Measurement
dev4 = qml.device("lightning.qubit", wires=qubits, shots=1000)
@qml.qnode(dev4)
def measurement(comb_state):
    qml.StatePrep(comb_state, wires=range(qubits))
    return qml.counts(all_outcomes=False)

counts = measurement(comb_state)
print(counts)  # Output: {'11110000': 500, '00001111': 500}


{'00111100': 1000}
